<a href="https://colab.research.google.com/github/41371131h-chi/114-1-/blob/main/HW4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==============================================================================
# 0. 環境設置與函式庫導入
# ==============================================================================
# --- 運行環境設定（請在 Colab Cell 中執行）---
!pip -q install gspread gspread_dataframe google-auth google-auth-oauthlib google-auth-httplib2 \
              gradio pandas beautifulsoup4 google-generativeai python-dateutil scikit-learn

In [2]:
import os, time, uuid, re, json, datetime
from datetime import datetime as dt, timedelta
from dateutil.tz import gettz
import pandas as pd
import gradio as gr
import requests
from bs4 import BeautifulSoup
import google.generativeai as genai

# Google Auth & Sheets
from google.colab import auth
import gspread
from gspread_dataframe import set_with_dataframe, get_as_dataframe
from google.auth.transport.requests import Request
from google.oauth2 import service_account
from google.auth import default

In [3]:
# TF-IDF (新增)
try:
    from sklearn.feature_extraction.text import TfidfVectorizer
except ImportError:
    print("⚠️ 未安裝 scikit-learn，TF-IDF 相關功能可能無法執行。請運行：!pip install scikit-learn")
    # 假裝 TfidfVectorizer 存在以避免程式碼崩潰
    class DummyTfidfVectorizer:
        def fit_transform(self, X): return None
        def get_feature_names_out(self): return []
    TfidfVectorizer = DummyTfidfVectorizer

In [4]:
# 認證與 Gemini 配置（請在 Colab Cell 中執行）
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

from google.colab import userdata
# 從 Colab Secrets 中獲取 API 金鑰
try:
    api_key = userdata.get('HW3')
    # 確保金鑰存在，否則 genai.configure 會報錯
    if not api_key:
        raise ValueError("Colab Secret 'HW3' is empty or not found.")

    genai.configure(api_key=api_key)
    print("✅ Gemini API Key 配置成功。")
except Exception as e:
    print(f"⚠️ Gemini API Key 配置失敗，請檢查 Colab Secrets 中的 'HW3'：{e}")

✅ Gemini API Key 配置成功。


In [5]:
# ==============================================================================
# 1. 全域變數與 Sheet/DataFrame 設置
# ==============================================================================

# 請確保這個 Sheet URL 存在且您有編輯權限
SHEET_URL = "https://docs.google.com/spreadsheets/d/1jR3qRQr2ZvWYKNuv8wen_-eTZWdc5a-LLvH7iymn2zw/edit?usp=sharing"
WORKSHEET_NAME = "工作表4" # 變更為更專注於爬蟲的名稱
TIMEZONE = "Asia/Taipei"

# Headers for Worksheets
CLIPS_HEADER = ["clip_id","url","selector","text","href","created_at","added_to_task"]
STATS_HEADER = ["keyword", "frequency", "score", "created_at"] # TF-IDF 統計表頭

In [6]:
# --- Google Sheet 初始化函式 ---
def ensure_spreadsheet(name):
    """確保試算表存在，若無則創建。"""
    try:
        sh = gc.open(name)
    except gspread.SpreadsheetNotFound:
        sh = gc.create(name)
    return sh

def ensure_worksheet(sh, title, header):
    """確保工作表存在且表頭正確。"""
    try:
        ws = sh.worksheet(title)
    except gspread.WorksheetNotFound:
        ws = sh.add_worksheet(title=title, rows="1000", cols=str(len(header)+5))
        ws.update([header])

    # 確保表頭正確
    data = ws.get_all_values()
    if not data or (data and data[0] != header):
        ws.clear()
        ws.update([header])
    return ws

sh = ensure_spreadsheet(WORKSHEET_NAME)
ws_clips = ensure_worksheet(sh, "web_clips", CLIPS_HEADER)
ws_stats = ensure_worksheet(sh, "tf-idf_stats", STATS_HEADER)

In [7]:
# ==============================================================================
# 2. DataFrame 讀寫與實用函式
# ==============================================================================

def tznow():
    return dt.now(gettz(TIMEZONE))

In [8]:
def read_df(ws, header):
    """讀取工作表為 DataFrame。"""
    df = get_as_dataframe(ws, evaluate_formulas=True, header=0)
    if df is None or df.empty:
        return pd.DataFrame(columns=header)
    df = df.fillna("")
    # 保證欄位齊全
    for c in header:
        if c not in df.columns:
            df[c] = ""
    # 移除不必要的欄位並保持順序
    df = df.reindex(columns=header, fill_value="")
    return df[header]

In [9]:
def write_df(ws, df, header):
    """將 DataFrame 寫入工作表。"""
    if df.empty:
        ws.clear()
        ws.update([header])
        return
    # 轉字串避免 gspread 型別問題
    df_out = df.copy()
    for c in df_out.columns:
        df_out[c] = df_out[c].astype(str)
    ws.clear()
    ws.update([header] + df_out.values.tolist())

In [10]:
def refresh_all():
    """從 Google Sheet 重新讀取 web_clips 和 tf-idf_stats。"""
    return (
        read_df(ws_clips, CLIPS_HEADER).copy(),
        read_df(ws_stats, STATS_HEADER).copy()
    )

clips_df, stats_df = refresh_all()

In [11]:
# --- 新增：寫入 AI 摘要至 Sheet 的函式 ---
def write_summary_to_sheet(keywords_str, summary_md):
    """將 Gemini 摘要寫入 ai_summary 工作表，將新紀錄放在最前面。"""
    global ws_summary

    # 讀取現有資料
    df_existing = read_df(ws_summary, SUMMARY_HEADER)

    new_row = pd.DataFrame([{
        "created_at": tznow().isoformat(),
        # 為了簡潔，將關鍵詞列表轉為字串儲存
        "keywords_used": keywords_str,
        "summary_report": summary_md
    }])

    # 合併現有資料和新資料 (將新資料放在最前面)
    df_updated = pd.concat([new_row, df_existing], ignore_index=True)

    # 只保留欄位順序
    df_updated = df_updated[SUMMARY_HEADER]

    write_df(ws_summary, df_updated, SUMMARY_HEADER)

    return len(df_updated)

In [12]:
# ==============================================================================
# 3. 爬蟲、TF-IDF 統計與 Gemini 摘要
# ==============================================================================

def fetch_and_store_clips(url, selector, mode, limit):
    """執行爬蟲，並將結果寫入 web_clips 工作表。"""
    global clips_df

    if not url or not selector:
        return "⚠️ URL 或 Selector 不可為空。", clips_df

    try:
        resp = requests.get(url, timeout=15, headers={"User-Agent":"Mozilla/5.0"})
        resp.raise_for_status()
    except Exception as e:
        return f"⚠️ 請求失敗：{e}", clips_df

    soup = BeautifulSoup(resp.text, "html.parser")
    nodes = soup.select(selector)
    rows = []

    limit = int(limit) if limit else 20

    for i, n in enumerate(nodes[:limit]):
        text = n.get_text(strip=True) if mode in ("text","both") else ""
        href = n.get("href") if mode in ("href","both") else ""

        if href and href.startswith("/"):
            from urllib.parse import urljoin
            href = urljoin(url, href)

        rows.append({
            "clip_id": str(uuid.uuid4())[:8], "url": url, "selector": selector,
            "text": text, "href": href, "created_at": tznow().isoformat(),
            "added_to_task": "" # 保留此欄位以便未來擴充
        })

    df = pd.DataFrame(rows, columns=CLIPS_HEADER)

    if not df.empty:
        # 寫入新的 web_clips 資料 (覆蓋)
        clips_df = df.copy()
        write_df(ws_clips, clips_df, CLIPS_HEADER)
    else:
        clips_df = pd.DataFrame(columns=CLIPS_HEADER)
        write_df(ws_clips, clips_df, CLIPS_HEADER)


    return f"✅ 成功擷取 {len(df)} 筆資料並寫入 web_clips。", clips_df

In [13]:
def calculate_tfidf_and_store(n_keywords):
    """從 web_clips 讀取文本，計算 TF-IDF，並將前 N 熱詞寫入 tf-idf_stats。"""
    global clips_df, ws_stats, stats_df

    texts = clips_df[clips_df['text'].astype(str).str.strip() != '']['text'].tolist()
    if not texts:
        return "⚠️ 爬蟲結果無有效文本可供統計。", "", pd.DataFrame(columns=STATS_HEADER)

    n_keywords = int(n_keywords)
    # 假設多數爬蟲內容為英文，使用英文停用詞
    vectorizer = TfidfVectorizer(stop_words='english', max_features=500, ngram_range=(1, 2))
    tfidf_matrix = vectorizer.fit_transform(texts)
    feature_names = vectorizer.get_feature_names_out()

    tfidf_scores = tfidf_matrix.max(axis=0).toarray().flatten()

    indices = tfidf_scores.argsort()[-n_keywords:][::-1]

    stats_rows = []
    _now = tznow().isoformat()
    for i in indices:
        stats_rows.append({
            "keyword": feature_names[i],
            "frequency": 0, # Placeholder (TF-IDF不直接是頻率)
            "score": round(tfidf_scores[i], 4),
            "created_at": _now
        })

    stats_df = pd.DataFrame(stats_rows, columns=STATS_HEADER)

    write_df(ws_stats, stats_df, STATS_HEADER)

    output_md = "### 🔥 關鍵詞 Top {} (TF-IDF Score)\n".format(n_keywords)
    for _, r in stats_df.iterrows():
        output_md += f"- **{r['keyword']}**：{r['score']:.4f}\n"

    return f"✅ 成功計算 TF-IDF 並將前 {n_keywords} 熱詞寫入 tf-idf_stats。", output_md, stats_df

In [14]:
# *** 修正此函式中的 `if not genai.api_key:` 檢查 ***
def generate_summary_and_insight(stats_df):
    """串接 Gemini API 生成 5 句洞察摘要與 120 字結論。"""

    # --- 修正後的檢查邏輯 ---
    # 檢查全局配置時使用的 api_key 變數是否存在或為空
    global api_key # 確保能存取在開頭從 Colab Secret 取得的 api_key 變數
    if not api_key:
        return "⚠️ Gemini API Key 未配置（全域變數 `api_key` 為空），無法生成摘要。", ""

    try:
        # 如果 genai.configure 成功，這裡就可以直接使用模型
        model = genai.GenerativeModel('gemini-2.5-pro')
    except Exception:
        # 如果模型加載失敗 (例如配置仍然無效)，則返回錯誤
        return "⚠️ Gemini 模型配置錯誤或金鑰無效。", ""

    if stats_df.empty:
        return "⚠️ 統計數據為空，無法生成摘要。", ""

    keywords_str = "\n".join([f"- {r['keyword']} (Score: {r['score']:.4f})" for _, r in stats_df.iterrows()])

    sys_prompt = (
        "你是一位專業的資料分析師。請根據提供的關鍵詞和分數，生成一份簡潔的報告。\n"
        "報告必須包含兩個部分：\n"
        "1. **五個關鍵洞察 (5 Key Insights)**：以五個獨立的句子條列出來，解釋這些關鍵詞在主題上的潛在意義。\n"
        "2. **結論 (Conclusion)**：一段約 120 字的總結，概括關鍵詞所顯示的主要趨勢或主題。\n"
        "請直接輸出 Markdown 格式。"
    )

    user_content = f"--- 關鍵詞分析結果 ---\n{keywords_str}"

    try:
        resp = model.generate_content(
            sys_prompt + "\n\n" + user_content,
            generation_config={"temperature": 0.5}
        )
        return "✅ Gemini 摘要生成完成。", resp.text
    except Exception as e:
        # 捕捉 API 請求時可能發生的錯誤 (如網路問題、API 限制等)
        return f"⚠️ Gemini 請求失敗：{e}", ""

In [15]:
def run_full_crawler_pipeline(url, selector, mode, limit, n_keywords):
    """執行完整自動化流程：爬蟲 -> 寫入 -> TF-IDF -> 寫入 -> Gemini 摘要 -> 寫入摘要。"""

    # *** 修正：將 global 聲明移動到函式內部最頂端 ***
    global clips_df, stats_df

    # 1. 爬蟲與寫入
    msg_crawl, new_clips_df = fetch_and_store_clips(url, selector, mode, limit)

    # 如果爬蟲失敗，直接返回錯誤
    if new_clips_df.empty:
        full_report = f"## 🤖 自動化流程執行報告\n\n### 步驟一：網頁爬蟲與資料寫入\n{msg_crawl}\n\n⚠️ **嚴重警告：爬蟲未擷取到任何文本！** TF-IDF 與 Gemini 分析無法進行。請檢查您的 URL 和 CSS Selector 是否正確。"
        # 這裡不需要重新賦值，因為 clips_df 在 fetch_and_store_clips 內部已經被修改為空 DataFrame
        return full_report, clips_df, stats_df

    # 2. TF-IDF 統計與寫入
    msg_tfidf, tfidf_md, stats_df_result = calculate_tfidf_and_store(n_keywords)

    # 3. Gemini 摘要生成
    msg_gemini_status, gemini_md = generate_summary_and_insight(stats_df_result)

    # --- 4. 新增：將摘要寫入 ai_summary 工作表 ---
    msg_summary_write = ""
    if not msg_gemini_status.startswith("⚠️"):
        try:
             # 提取 TF-IDF 關鍵詞字串
             keywords_str = ", ".join(stats_df_result['keyword'].tolist())
             write_summary_to_sheet(keywords_str, gemini_md)
             msg_summary_write = f"✅ 成功將 AI 摘要寫入 ai_summary。"
        except Exception as e:
             msg_summary_write = f"⚠️ 寫入 AI 摘要至 Sheet 失敗: {e}"
    else:
        msg_summary_write = "⚠️ AI 摘要生成失敗，故未寫入 Sheet。"

    # 合併輸出
    full_report = (
        f"## 🤖 自動化流程執行報告\n\n"
        f"### 步驟一：網頁爬蟲與資料寫入\n{msg_crawl}\n\n"
        f"### 步驟二：詞頻與關鍵詞統計 (TF-IDF)\n{msg_tfidf}\n\n"
        f"{tfidf_md}\n\n"
        f"### 步驟三：AI 洞察摘要 (Gemini)\n{msg_gemini_status}\n\n"
        f"### 步驟四：AI 摘要寫入 Sheet\n{msg_summary_write}\n\n"
        f"--- AI 報告 ---\n"
        f"{gemini_md}"
    )

    # 這裡不需要再次呼叫 refresh_all()，因為 clips_df 和 stats_df 在前面的步驟中已被寫入並更新
    # 直接使用全域變數 clips_df, stats_df 作為返回值
    return full_report, clips_df, stats_df

In [16]:
# ==============================================================================
# 4. Gradio 介面（僅限爬蟲分析）
# ==============================================================================

def _refresh_dataframes():
    """僅刷新 web_clips 和 tf-idf_stats 給 Gradio"""
    global clips_df, stats_df
    clips_df, stats_df = refresh_all()
    return clips_df, stats_df

clips_df, stats_df = refresh_all() # 確保介面初始值最新

with gr.Blocks(title="純爬蟲分析與 AI 摘要（Google Sheet＋Gradio）") as demo:
    gr.Markdown("# 🕷️ 網頁爬蟲 → TF-IDF 關鍵詞分析 → AI 洞察摘要")
    gr.Markdown("此工具會自動執行：**爬蟲 → 寫入 Sheet (`web_clips`) → 讀取文本並 TF-IDF 統計 → 寫入 Sheet (`tf-idf_stats`) → Gemini 生成摘要**。")

    with gr.Row():
        btn_refresh = gr.Button("🔄 重新整理 Sheet 資料（Web Clips & Stats）")

    with gr.Tab("Crawler Analysis"):
        with gr.Row():
            url = gr.Textbox(label="目標 URL", placeholder="https://example.com/news", value="https://www.bbc.com/news")
            selector = gr.Textbox(label="CSS Selector", placeholder="h2 a", value="h2 a")

        with gr.Row():
            mode = gr.Radio(["text","href","both"], value="text", label="擷取內容")
            limit = gr.Number(value=20, precision=0, label="最多擷取幾筆")
            n_keywords = gr.Number(value=10, precision=0, label="TF-IDF 輸出熱詞數 (N)")

        btn_run_crawler = gr.Button("🚀 一鍵執行爬蟲與分析流程")

        out_report = gr.Markdown("## 報告輸出區")

        with gr.Row():
            with gr.Column():
                gr.Markdown("### 原始爬蟲結果 (web_clips)")
                grid_clips = gr.Dataframe(value=clips_df, label="web_clips 最新資料", interactive=False)
            with gr.Column():
                gr.Markdown("### TF-IDF 統計結果 (tf-idf_stats)")
                grid_stats = gr.Dataframe(value=stats_df, label="TF-IDF 最新統計", interactive=False)


    # === 綁定動作 ===
    btn_refresh.click(_refresh_dataframes, outputs=[grid_clips, grid_stats])

    btn_run_crawler.click(
        run_full_crawler_pipeline,
        inputs=[url, selector, mode, limit, n_keywords],
        outputs=[out_report, grid_clips, grid_stats]
    )

In [ ]:
demo.launch(debug=True) # 加上 debug=True

It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://39b6dde4c978d7e8d9.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
